#Long Read RNA Sequencing Analysis Pipeline (Requires Pre-Built Indices)


A pipeline to analyze Oxford Nanopore and PacBio third-generation long transcriptomic sequencing reads

*Theodore Nelson (tmn2126)* - Columbia University Irving Medical Center


##Parameter Input and User Instructions

Please define where the file structure is within your Google Drive:

<ul type=disc>
<li><b>PIPELINE_FILE_PATH</b>: file path to location of long-read RNA sequencing analysis pipeline within your Google Drive/general file system - required for most applications. This must be defined and initialized first.</li>
</ul>

In [ ]:
%env PIPELINE_FILE_PATH=/content/drive/MyDrive/long-read-sequencing-pipeline

Please modify the following parameters within the code box below to fit your own study requirements:  

<li><b>ACC</b>: Run accession number for reads within the [European Nucleotide Archive](https://www.ebi.ac.uk/ena/browser/) (SRR...) or file path to location of long-read RNA sequencing data within your Google Drive/general file system - required for most applications</li>
<li><b>PLATFORM_FREE</b>: choose either `map-ont` (Nanopore reads) or `map-pac` (PacBio reads) - required for minimap2 featherweight version</li>
<li><b>INDEX_FILE_PATH</b>: file path to location of reference genome (e.g. .FASTA) within your Google Drive/general file system - required for most applications</li>
<li><b>ANNOTATION_FILE_PATH</b>: file path to location of reference annotation (e.g. .GTF) within your Google Drive/general file system - required for most applications</li>
<li><b>PARTITIONED_INDEX_FILE_PATH</b>: file path to location of partioned reference annotation (e.g. .idx) within your Google Drive/general file system - required for minimap2 featherweight version</li>
<li><b>CHROMOSOME</b>: Name of Chromosome of Interest matching the name of the Chromosome within your Reference Annotation - required for svist4get</li>
<li><b>CHROMOSOME_START</b>: Starting Location of Interest on the Chromosome - required for svist4get</li>
<li><b>CHROMOSOME_FINISH</b>: Ending Location of Interest on the Chromosome - required for svist4get</li>
<li><b>REGION_NAME</b>: Gene Name (does not need to match annotation file) - required for svist4get</li>
<li><b>HUB_KEYWORD</b>: Short Keyword for your UCSC Track Hub - required for MakeHub</li>
<li><b>HUB_NAME</b>: Longer Title for your UCSC Track Hub - required for MakeHub</li>
<li><b>HUB_EMAIL</b>: Email for your UCSC Track Hub (if you publish your track hub then this email will be public) - required for MakeHub</li>


In [ ]:
%env ACC=ERR1951293  
%env PLATFORM_FREE=map-ont
%env INDEX_FILE_PATH=${PIPELINE_FILE_PATH}/prebuilt_indices/mm39.fa
%env ANNOTATION_FILE_PATH=${PIPELINE_FILE_PATH}/prebuilt_indices/mm39.ncbiRefSeq.gtf
%env PARTITIONED_INDEX_FILE_PATH=${PIPELINE_FILE_PATH}/prebuilt_indices/mm39.idx
%env CHROMOSOME=chr12 
%env CHROMOSOME_START=116533435 
%env CHROMOSOME_FINISH=116536513
%env REGION_NAME=LINC00173
%env HUB_KEYWORD=lnc173
%env HUB_NAME="293T LINC00173"
%env HUB_EMAIL=tmn2126@columbia.edu

##Mounting your Google Drive

This step allows for permeanent storage of your bioinformatics analysis in Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

##Managing Software via BioConda

BioConda is a software environment and package manager, providing acess to over 8,000 different software packages related to bioinformatics (documentation: [BioConda](https://bioconda.github.io/user/install.html) and [Managing Environments via Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html)). 

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

##Kingfisher: fast and flexible program for procurement of sequence files - installation 

The Kingfisher program allows for sequence files to be downloaded from the European Nucleotide Archive (documentation: https://github.com/wwood/kingfisher-download).

In [ ]:
! git clone https://github.com/MakeTheBrainHappy/kingfisher-download

In [ ]:
! conda env update -n base --file kingfisher-download/kingfisher.yml

In [ ]:
! conda install -c rpetit3 aspera-connect -y

In [ ]:
! wget -qO- https://download.asperasoft.com/download/sw/connect/3.9.8/ibm-aspera-connect-3.9.8.176272-linux-g2.12-64.tar.gz | tar xvz

this command will pop up with an error message; please disregard

In [ ]:
! ./ibm-aspera-connect-3.9.8.176272-linux-g2.12-64.sh

##Kingfisher: fast and flexible program for procurement of sequence files - usage 


The Kingfisher program allows for sequence files to be downloaded from the European Nucleotide Archive (documentation: https://github.com/wwood/kingfisher-download).

In [ ]:
! cd $PIPELINE_FILE_PATH/fastq ; /content/kingfisher-download/bin/kingfisher get -r $ACC -m ena-ascp aws-http prefetch

In [ ]:
! cd $PIPELINE_FILE_PATH/fastq ; gunzip *.gz

In [ ]:
! cd $PIPELINE_FILE_PATH/fastq ; mv ${ACC}_1.fastq $ACC.fastq

##FastQC: A quality control tool for high throughput sequence data - installation

FastQC is a program designed to spot potential problems in high througput sequencing datasets ([documentation](https://github.com/s-andrews/FastQC)).

In [ ]:
! conda install -c bioconda fastqc -y

##FastQC: A quality control tool for high throughput sequence data - usage

FastQC is a program designed to spot potential problems in high througput sequencing datasets ([documentation](https://github.com/s-andrews/FastQC)).

In [ ]:
! fastqc $PIPELINE_FILE_PATH/fastq/$ACC.fastq --outdir $PIPELINE_FILE_PATH/fastqc

##minimap2: A versatile pairwise aligner for genomic and spliced nucleotide sequences (Google Colab Pro required) - installation

minimap2 is a long-read sequencing aligner (documentation: https://github.com/lh3/minimap2). 

In [ ]:
! conda install -c bioconda minimap2 -y

##minimap2: A versatile pairwise aligner for genomic and spliced nucleotide sequences (Google Colab Pro required) - usage

minimap2 is a long-read sequencing aligner (documentation: https://github.com/lh3/minimap2). 

Google Colab will typically provide virtual machines with 12 GB of RAM. minimap2 in the case of the human genome usually requires more than 12 GB of RAM. Therefore this command will fail to execute within the free version of Google Colab. However, this pipeline also implements a less memory-intensive version of minimap2 for free tier users. 

In [ ]:
! eval minimap2 -ax splice $INDEX_FILE_PATH $PIPELINE_FILE_PATH/fastq/$ACC.fastq > $PIPELINE_FILE_PATH/sam/$ACC.sam

##minimap2 featherweight alignment - installation

The minimap2 featherweight implementation works by splitting the reference genome into four parts with minimal loss in alignment accuracy (documentation: https://doi.org/10.1038/s41598-019-40739-8).

In [ ]:
! wget https://github.com/hasindu2008/minimap2-arm/archive/v0.1.tar.gz
! tar xvf v0.1.tar.gz && cd minimap2-arm-0.1 && make

In [ ]:
! chmod u+x /content/minimap2-arm-0.1/misc/idxtools/divide_and_index.sh

In [ ]:
! apt-get install bc

##minimap2 featherweight alignment - partioning 

The minimap2 featherweight implementation works by splitting the reference genome into four parts with minimal loss in alignment accuracy (documentation: https://doi.org/10.1038/s41598-019-40739-8). This command will only need to be run once as long as the partioned reference index is not removed from the directory. 

In [ ]:
! eval /content/minimap2-arm-0.1/misc/idxtools/divide_and_index.sh $INDEX_FILE_PATH 4 $PARTITIONED_INDEX_FILE_PATH /content/minimap2-arm-0.1/minimap2 $PLATFORM_FREE

##minimap2 featherweight alignment - usage

The minimap2 featherweight implementation works by splitting the reference genome into four parts with minimal loss in alignment accuracy (documentation: https://doi.org/10.1038/s41598-019-40739-8).

In [ ]:
! eval /content/minimap2-arm-0.1/minimap2 -a -x $PLATFORM_FREE $PARTITIONED_INDEX_FILE_PATH $PIPELINE_FILE_PATH/fastq/$ACC.fastq --multi-prefix tmp > $PIPELINE_FILE_PATH/sam/$ACC.sam

##samtools: Reading/writing/editing/indexing/viewing SAM/BAM/CRAM format - installation

samtools allows for manipulation of high-throughput sequencing data (documentation: http://www.htslib.org/) 


In [ ]:
! conda install -c bioconda samtools -y

##samtools: Reading/writing/editing/indexing/viewing SAM/BAM/CRAM format - usage

samtools allows for manipulation of high-throughput sequencing data (documentation: http://www.htslib.org/) 


In [ ]:
! samtools view -S -b $PIPELINE_FILE_PATH/sam/$ACC.sam > $PIPELINE_FILE_PATH/bam/$ACC.bam 

In [ ]:
! samtools sort $PIPELINE_FILE_PATH/bam/$ACC.bam -o $PIPELINE_FILE_PATH/bam/$ACC.sorted.bam  

In [ ]:
! samtools index $PIPELINE_FILE_PATH/bam/$ACC.sorted.bam 

##TranscriptClean: correct mismatches, microindels, and noncanonical splice junctions - installation

TranscriptClean is a command-line program which corrects long-read mismatches, microindels and noncanonical splice junctions (documentation: https://github.com/mortazavilab/TranscriptClean). 

In [ ]:
! conda install -c bioconda pyfasta pyranges samtools -y

In [ ]:
! wget https://github.com/mortazavilab/TranscriptClean/archive/refs/tags/v2.0.3.tar.gz
! tar xvf v2.0.3.tar.gz 

##TranscriptClean: correct mismatches, microindels, and noncanonical splice junctions - usage

TranscriptClean is a command-line program which corrects long-read mismatches, microindels and noncanonical splice junctions (documentation: https://github.com/mortazavilab/TranscriptClean). 

In [ ]:
! eval python /content/TranscriptClean-2.0.3/TranscriptClean.py --sam $PIPELINE_FILE_PATH/sam/$ACC.sam --genome $INDEX_FILE_PATH --outprefix $PIPELINE_FILE_PATH/transcriptclean/$ACC

##featureCounts: an efficient general purpose program for assigning sequence reads to genomic features - installation

featureCounts is a read summarization program suitable for counting reads generated from either RNA or genomic DNA sequencing experiments (documentation: http://subread.sourceforge.net/). 

In [ ]:
! conda install -c bioconda subread -y

##featureCounts: an efficient general purpose program for assigning sequence reads to genomic features - usage

featureCounts is a read summarization program suitable for counting reads generated from either RNA or genomic DNA sequencing experiments (documentation: http://subread.sourceforge.net/). 

In [ ]:
! eval featureCounts -M -O -T 24 -L -a $ANNOTATION_FILE_PATH -t exon -g gene_id -o $PIPELINE_FILE_PATH/featureCounts/$ACC.txt $PIPELINE_FILE_PATH/bam/$ACC.bam 

##TAMA: Transcriptome Annotation by Modular Algorithms (Google Colab Pro required) - installation



TAMA allows for the construction of a polished transcriptome by collapsing runs into individual transcripts (documentation: https://github.com/GenomeRIK/tama/wiki). 

In [ ]:
! git clone https://github.com/MakeTheBrainHappy/tama

In [ ]:
! python -m pip install Bio

In [ ]:
! pip install pysam

##TAMA: Transcriptome Annotation by Modular Algorithms (Google Colab Pro required) - usage

TAMA allows for the construction of a polished transcriptome by collapsing runs into individual transcripts (documentation: https://github.com/GenomeRIK/tama/wiki). 

In [ ]:
! eval python tama/tama_collapse.py -s $PIPELINE_FILE_PATH/bam/$ACC.sorted.bam -f $INDEX_FILE_PATH -p $PIPELINE_FILE_PATH/tama/$ACC.polished -b BAM -rm low_mem

##svist4get: a simple visualization tool for genomic tracks from sequencing experiments - installation

svist4get allows you to view read coverage at a defined region on a chromosome (documentation: https://bitbucket.org/artegorov/svist4get/src/master/)

In [ ]:
! sudo apt-get install bedtools

In [ ]:
! apt-get update

In [ ]:
! apt-get install libmagickwand-dev

In [ ]:
! cp -r $PIPELINE_FILE_PATH/svist4get/policy_revised.xml /etc/ImageMagick-6/policy.xml

In [ ]:
! python3 -m pip install svist4get

##svist4get: a simple visualization tool for genomic tracks from sequencing experiments - usage

svist4get allows you to view read coverage at a defined region on a chromosome (documentation: https://bitbucket.org/artegorov/svist4get/src/master/)

In [ ]:
! bedtools genomecov -ibam $PIPELINE_FILE_PATH/bam/$ACC.sorted.bam -bg > $PIPELINE_FILE_PATH/bed/$ACC.sorted.bedgraph

In [ ]:
! eval svist4get -bg $PIPELINE_FILE_PATH/bed/$ACC.sorted.bedgraph -gtf $ANNOTATION_FILE_PATH -fa $INDEX_FILE_PATH -bl Long-Read Coverage -w $CHROMOSOME $CHROMOSOME_START $CHROMOSOME_FINISH -it "$REGION_NAME" -o $PIPELINE_FILE_PATH/svist4get/$ACC

##Pistis: Quality control plotting for long reads - installation

Pistis generates long-read specific quality control graphs, including a plot demonstrating read alignment percentage to the reference genome (documentation: https://github.com/mbhall88/pistis)

In [ ]:
! pip3 install pistis

##Pistis: Quality control plotting for long reads - usage

Pistis generates long-read specific quality control graphs, including a plot demonstrating read alignment percentage to the reference genome (documentation: https://github.com/mbhall88/pistis)

In [ ]:
! pistis -f $PIPELINE_FILE_PATH/fastq/$ACC.fastq -b $PIPELINE_FILE_PATH/bam/$ACC.sorted.bam  -o $PIPELINE_FILE_PATH/pistis/$ACC.pdf

##MakeHub: Fully automated generation of UCSC assembly hubs - installation

MakeHub is a command line tool for the fully automatic generation of of track data hubs for visualizing genomes with the UCSC genome browser (documentation: https://github.com/Gaius-Augustus/MakeHub).

In [ ]:
! python3.7 -m pip install biopython

In [ ]:
! sudo apt install samtools

In [ ]:
! sudo apt install augustus augustus-data augustus-doc

##MakeHub: Fully automated generation of UCSC assembly hubs - usage

MakeHub is a command line tool for the fully automatic generation of of track data hubs for visualizing genomes with the UCSC genome browser (documentation: https://github.com/Gaius-Augustus/MakeHub).

In [ ]:
! chmod 755 $PIPELINE_FILE_PATH/makehub/make_hub.py

In [ ]:
! eval $PIPELINE_FILE_PATH/makehub/make_hub.py -l $HUB_KEYWORD -L $HUB_NAME -g $INDEX_FILE_PATH -e \
  $HUB_EMAIL -a $ANNOTATION_FILE_PATH -b $PIPELINE_FILE_PATH/bam/$ACC.sorted.bam -o $PIPELINE_FILE_PATH/makehub/

##MultiQC: Aggregate results from bioinformatics analyses across many samples into a single report - installation



MultiQC is a program which allows you to combine reports for as many samples as you wish ([documentation](https://multiqc.info/docs/))

In [ ]:
! pip install multiqc

##MultiQC: Aggregate results from bioinformatics analyses across many samples into a single report - usage

MultiQC is a program which allows you to combine reports for as many samples as you wish ([documentation](https://multiqc.info/docs/))

In [ ]:
! multiqc $PIPELINE_FILE_PATH -o $PIPELINE_FILE_PATH/multiqc